## day52_pyspark_Window_function_&_set_operators


### ⚙️ Window Functions = Analytical Functions

**Definition:**
A window function performs a calculation across a set of rows that are related to the current row — called a *window*.

Examples include:

* `rank()`
* `dense_rank()`
* `row_number()`
* `sum() over()`, `avg() over()` (for running totals or moving averages)


In [0]:
from pyspark.sql.functions import *
Data = (("James", "Sales", 3000), \
              ("Michael", "Sales", 4600), \
              ("Robert", "Sales", 4100), \
              ("Maria", "Finance", 3000), \
              ("James", "Sales", 3000), \
              ("Scott", "Finance", 3300), \
              ("Jen", "Finance", 3900), \
              ("Jeff", "Marketing", 3000), \
              ("Kumar", "Marketing", 2000), \
              ("Saif", "Sales", 4100) \
              )

columns = ["employee_name", "department", "salary"]
df = spark.createDataFrame(data=Data, schema=columns)
df.printSchema()
df.display()

In [0]:
df.createOrReplaceTempView("df_v")

In [0]:
spark.sql("""select a.*,
          rank() over( order by salary desc) as rank,
          dense_rank() over( order by salary desc) as dense_rank,
          row_number() over( order by salary desc) as row_number
          from df_v a """).display()

In [0]:
from pyspark.sql.window import Window
w = Window.orderBy(col("salary").desc())
df.withColumn("rank", rank().over(w)) \
  .withColumn("dense_rank", dense_rank().over(w)) \
  .withColumn("row_number", row_number().over(w)).display()

#----------both are same --- (use '\' to break the line  or use ( df. .... (display() end)) -------------

(df.withColumn("rank", rank().over(w))
  .withColumn("dense_rank", dense_rank().over(w))
  .withColumn("row_number", row_number().over(w)).display() ) 

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from datetime import date

customer_data = [
    (1, "Alice", date(2023, 2, 10)),
    (2, "Bob", date(2023, 2, 15)),
    (3, "Charlie", date(2023, 3, 20)),
    (1, "Alice", date(2023, 1, 10)),      # duplicate
    (4, "Diana", date(2023, 4, 25)),
    (2, "Bob", date(2023, 2, 15)),        # duplicate
    (5, "Eve", date(2023, 5, 30))
]

customer_schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("create_date", DateType(), True)
])

customer_df = spark.createDataFrame(data=customer_data, schema=customer_schema)
display(customer_df)


In [0]:
# row_number() 

customer_df.withColumn('row_number', row_number().over(Window.partitionBy('emp_id').orderBy(col('create_date').desc()))).filter("row_number = 1").display()

In [0]:
# Rank the customers by the

w = Window.partitionBy('emp_id').orderBy(col('create_date').desc())
customer_df.withColumn('rank', rank().over(w)).filter("rank = 1").display()

In [0]:
customer_df.createOrReplaceTempView("customer_v")

In [0]:
spark.sql("""select * from (select a.*, row_number() over (partition by emp_id order by create_date desc) as row_number from customer_v a) where row_number = 1 """).display()

Got it—think of **set operations** as “combine or compare two result sets without caring about relational keys.” Same rules as SQL: **same number of columns, compatible types.** In PySpark we usually work with **DataFrames** and use `union`, `intersect`, and `except`-style ops (plus a few extras).

# 1) Quick cheatsheet

* **UNION ALL** → stack rows from both (keep duplicates)
  PySpark: `df1.union(df2)`
* **UNION (distinct)** → stack rows, **remove duplicates**
  PySpark: `df1.union(df2).distinct()`
###  most:->  in pyspark union and unionall both are same   ( gave a unionall result, not remove distnct vale) ) 
* **UNION BY NAME** → match columns by name (order can differ)
  PySpark: `df1.unionByName(df2, allowMissingColumns=True)`
* **INTERSECT** → rows common to both
  PySpark: `df1.intersect(df2)`
* **EXCEPT / MINUS** → rows in A **not** in B
  PySpark: `df1.exceptAll(df2)` (keeps dup counts) or `df1.exceptAll(df2).distinct()` if you want set-like
* **SUBTRACT** (RDD-era alias) → older style; prefer `exceptAll`

> Tip: If schemas differ only by **column order**, use `unionByName`. If datatypes differ (e.g., `int` vs `decimal`), **cast** first.

---

# 2) Minimal example (same schema)

```python
from pyspark.sql import Row, functions as F, types as T

df1 = spark.createDataFrame([
    Row(city="Toronto",  product="A", margin=0.22, demand=120),
    Row(city="Toronto",  product="B", margin=0.18, demand= 80),
    Row(city="Montreal", product="A", margin=0.25, demand= 60),
    Row(city="Calgary",  product="C", margin=0.15, demand= 50),  # unique to df1
])

df2 = spark.createDataFrame([
    Row(city="Toronto",  product="A", margin=0.22, demand=120),  # duplicate across sets
    Row(city="Vancouver",product="B", margin=0.20, demand= 70),  # unique to df2
    Row(city="Montreal", product="A", margin=0.25, demand= 60),
    Row(city="Toronto",  product="B", margin=0.18, demand= 80),
])
```

**UNION ALL** (stack everything):

```python
df_union_all = df1.union(df2)
```

**UNION (distinct)**:

```python
df_union = df1.union(df2).distinct()
```

**INTERSECT** (common rows only):

```python
df_common = df1.intersect(df2)
```

**EXCEPT** (rows in df1 that are not in df2):

```python
df_only_in_df1 = df1.exceptAll(df2).distinct()
```

---

# 3) “Compare by condition” (your “city/margins/demand” ask)

Often you don’t want full-row equality; you want **conditional** comparisons (e.g., same city+product, but margin/demand differs). That’s when a **JOIN** + filters is better than pure set ops.

**Find rows where df1 vs df2 differ on margin or demand for the same (city, product):**

```python
keys = ["city", "product"]
cmp = (df1.alias("a")
         .join(df2.alias("b"), on=keys, how="full_outer")
         .select(
            F.coalesce(F.col("a.city"), F.col("b.city")).alias("city"),
            F.coalesce(F.col("a.product"), F.col("b.product")).alias("product"),
            F.col("a.margin").alias("margin_a"),
            F.col("b.margin").alias("margin_b"),
            F.col("a.demand").alias("demand_a"),
            F.col("b.demand").alias("demand_b"),
         ))

# differences only (tolerant to small float noise with a threshold)
diff = cmp.where(
    (F.col("margin_a").isNull() != F.col("margin_b").isNull()) |
    (F.col("demand_a").isNull() != F.col("demand_b").isNull()) |
    (F.abs(F.col("margin_a") - F.col("margin_b")) > F.lit(1e-9)) |
    (F.col("demand_a") != F.col("demand_b"))
)
```

**Classify results** (only in df1, only in df2, or in both but changed):

```python
result = (cmp
    .withColumn("presence",
        F.when(F.col("margin_b").isNull() & F.col("demand_b").isNull(), "only_in_df1")
         .when(F.col("margin_a").isNull() & F.col("demand_a").isNull(), "only_in_df2")
         .otherwise("in_both"))
    .withColumn("changed",
        (F.abs(F.col("margin_a") - F.col("margin_b")) > 1e-9) |
        (F.col("demand_a") != F.col("demand_b")))
)
```

> Use this pattern to answer questions like: “Compare two deductions by **city** based on **margin/demand conditions** and show differences.”

---

# 4) Handling duplicates intentionally

If duplicates matter (e.g., two identical rows indicate quantity), use the `*All` variants:

* `intersectAll` keeps min duplicate counts
* `exceptAll` removes with multiplicity
* `union` already behaves like **UNION ALL**

Example:

```python
common_with_dups = df1.intersectAll(df2)
only_in_a_with_dups = df1.exceptAll(df2)
```

---

# 5) Different column order or missing columns

```python
# Order differs or df2 missing a column -> align by name and fill nulls for missing
df_aligned = df1.unionByName(df2, allowMissingColumns=True)
```

If types differ, cast:

```python
df1_cast = df1.select(
    F.col("city"),
    F.col("product"),
    F.col("margin").cast("double").alias("margin"),
    F.col("demand").cast("int").alias("demand"),
)
```

---

# 6) When to use set ops vs joins

* **Set ops** → you care about **whole-row equality** (same all columns).
* **Join + filters** → you care about **keyed comparisons** and **per-column deltas** (your “margin/demand by city” case).

---

# 7) Tiny SQL equivalents (for reference)

```sql
-- UNION ALL
SELECT * FROM t1
UNION ALL
SELECT * FROM t2;

-- UNION (distinct)
SELECT * FROM t1
UNION
SELECT * FROM t2;

-- INTERSECT
SELECT * FROM t1
INTERSECT
SELECT * FROM t2;

-- EXCEPT / MINUS (engine-specific keyword)
SELECT * FROM t1
EXCEPT
SELECT * FROM t2;
```

---

If you share a small sample of your **two DataFrames** (columns + 5–10 rows), I’ll plug them into the exact patterns above and hand you the precise PySpark code for your case.


In [0]:
simpleData = [(1, "James", "Sales", "NY", 90000), \
              (2, "Michael", "Sales", "NY", 86000), \
              (3, "Robert", "Sales", "CA", 81000), \
              (4, "Maria", "Finance", "CA", 90000), \
              (1, "James", "Sales", "NY", 90000)
              ]

columns = ["emp_id", "employee_name", "department", "state", "salary"]
df1 = spark.createDataFrame(data=simpleData, schema=columns)

simpleData2 =   [ (1, "James", "Sales", "NY", 90000), \
                (1, "James", "Sales", "NY", 90000), \
                (2, "Michael", "Sales", "NY", 86000), \
                (2, "Michael", "Sales", "NY", 86000),\
                (5, "Sonja", "Sales", "OH", 45000), \
                (6, "Randy H", "Finance", "NJ", 40000)
                ]

columns = ["emp_id", "employee_name", "department", "state", "sal"]
df2 = spark.createDataFrame(data=simpleData2, schema=columns)

df1.display()
df2.display()

In [0]:
df1.createOrReplaceTempView("df1_v1")
df2.createOrReplaceTempView("df2_v2")

spark.sql("""select * from df1_v1 
          union 
          select  * from df2_v2""").display()

spark.sql("""select * from df1_v1 
          union all
          select  * from df2_v2""").display()

In [0]:
# in pyspark union and unionall both are same  ( 00:47)

df1.union(df2).display() # In pyspark union is union all
df1.unionAll(df2).display()

#### if you apply quick value the apply distinct() ( bothe union and unionall)

In [0]:
df1.unionAll(df2).distinct().display()

# ------------both are same

df1.union(df2).distinct().display()

### or same ways use dropDuplicates() method

In [0]:
### or same ways use dropDuplicates() method

df1.union(df2).dropDuplicates().display()
# df1.union(df2).display()  

In [0]:
# intersect

df1.intersect(df2).display()

In [0]:
# intersect

df2.intersect(df1).display()

In [0]:
df1.display()
df2.display()


In [0]:
df1.intersect(df2).display()
df1.intersectAll(df2).display()

Excellent — what you’ve shown in the image and explanation is **100% correct and practical** for PySpark’s `intersect()` vs `intersectAll()` behavior.

Here’s a clear, short explanation (in both **English and Gujarati**) for your notes or interview:

---

### 🧩 **English Explanation**

**Purpose:**
To find *common rows* between two DataFrames, similar to “intersection” in set theory.

**Syntax:**

```python
df1.intersect(df2).show()
df1.intersectAll(df2).show()
```

**Difference:**

* `intersect()` → Returns **distinct** common rows (removes duplicates).
* `intersectAll()` → Returns **all** common rows, including duplicates.

**Example Summary:**
If `df1` and `df2` both have rows `[1,1,2,3]` and `[1,2,2,3]`:

* `df1.intersect(df2)` → `[1,2,3]`
* `df1.intersectAll(df2)` → `[1,2,2,3]`

**Use Case:**
To identify matching records between two datasets — e.g., matching transactions, duplicate orders, or identical records after transformation.

---

### 🪔 **Gujarati Explanation**

**ઉદ્દેશ્ય:**
બે DataFrame વચ્ચેના સામાન્ય રો (common rows) શોધવા માટે, બિલકુલ *intersection* જેવી set theory મુજબ.

**Syntax:**

```python
df1.intersect(df2).show()
df1.intersectAll(df2).show()
```

**ફરક:**

* `intersect()` → ફક્ત **unique common rows** આપે (ડુપ્લિકેટ દૂર કરે).
* `intersectAll()` → **બધા common rows**, ડુપ્લિકેટ સહિત આપે.

**ઉદાહરણ:**
જો df1 = `[1,1,2,3]` અને df2 = `[1,2,2,3]` હોય, તો

* `df1.intersect(df2)` → `[1,2,3]`
* `df1.intersectAll(df2)` → `[1,2,2,3]`

**ઉપયોગ:**
બે ટેબલ વચ્ચે મેળ ખાતા રેકોર્ડ શોધવા માટે — જેમ કે સમાન ટ્રાન્ઝેક્શન, ડુપ્લિકેટ ઓર્ડર, અથવા પરિવર્તન પછી એકસરખા રેકોર્ડ.




### ExceptAll (Subtract) -- > minus
###### except and Subtract bothare same
#####   *returns row from the first DateFrame that are notpresent in the second dataframe.


In [0]:
simpleData = [(1, "James", "Sales", "NY", 90000), \
              (2, "Michael", "Sales", "NY", 86000), \
                  (2, "Michael", "Sales", "NY", 86000), \
              (3, "Robert", "Sales", "CA", 81000), \
              (4, "Maria", "Finance", "CA", 90000), \
              (1, "James", "Sales", "NY", 90000)
              ]

columns = ["emp_id", "employee_name", "department", "state", "salary"]
df1 = spark.createDataFrame(data=simpleData, schema=columns)

simpleData2 =   [ (1, "James", "Sales", "NY", 90000), \
                (1, "James", "Sales", "NY", 90000), \
                (2, "Michael", "Sales", "NY", 86000), \
                (2, "Michael", "Sales", "NY", 86000),\
                (5, "Sonja", "Sales", "OH", 45000), \
                (6, "Randy H", "Finance", "NJ", 40000)
                ]

columns = ["emp_id", "employee_name", "department", "state", "sal"]
df2 = spark.createDataFrame(data=simpleData2, schema=columns)

df1.display()
df2.display()

In [0]:
df1.exceptAll(df2).display()

df2.exceptAll(df1).display()

In [0]:
df2.exceptAll(df1).display()
#---------both are same-----------
df2.subtract(df1).display()

In [0]:
Name another data from DF2. If data different columns altogether. But still if you want to perform you can still do by union by name. So what it does is exactly. So first you try to see there are many matched columns between 2 database if matched columns if not matching displays null values in the specific cost. I will show you the example. Let me show you

### unionByName

Exactly — you’re talking about **`unionByName()`** in PySpark. Let’s make this concept very clear with example and Gujarati translation.

---

### 🧩 **English Explanation — `unionByName()`**

**Purpose:**
When two DataFrames have **different column names or order**, but you still want to combine them, you can use `unionByName()` instead of `union()`.

**Syntax:**

```python
df1.unionByName(df2).show()
```

**Optional Parameter:**

```python
df1.unionByName(df2, allowMissingColumns=True).show()
```

**What it does:**

* It matches columns **by name**, not by position.
* If a column is missing in one DataFrame, PySpark automatically fills it with **NULL**.

---

### **Example:**

```python
# DataFrame 1
df1 = spark.createDataFrame([
    (1, "John", 5000)
], ["id", "name", "salary"])

# DataFrame 2
df2 = spark.createDataFrame([
    ("Doe", 2, "Finance")
], ["name", "id", "dept"])

# Union by name (filling missing columns with NULL)
df_result = df1.unionByName(df2, allowMissingColumns=True)
df_result.show()
```

**Output:**

| id | name | salary | dept    |
| -- | ---- | ------ | ------- |
| 1  | John | 5000   | null    |
| 2  | Doe  | null   | Finance |

✅ **Explanation:**

* Columns are matched **by name**, not order.
* Missing columns (`dept` in df1, `salary` in df2) are filled with **null**.

**Use Case:**
Helpful when combining datasets from different systems or APIs where column structures differ slightly.



In [0]:
data1 = [(1, "Alice", 1000), (2, "Bob", 1500)]
columns1 = ["id", "name", "salary"]
df_a = spark.createDataFrame(data1, columns1)

data2 = [("Charlie", 3, 2000,'Java'), ("David", 4, 2500,'Python')]
columns2 = ["name", "id", "salary","language"]
df_b = spark.createDataFrame(data2, columns2)

df_a.display()
df_b.display()

In [0]:
df_a.union(df_b).display()

In [0]:
df_a.select('name', 'salary','id').union(df_b.select('name', 'salary','id')).display()

In [0]:
df_a.unionByName(df_b,allowMissingColumns=False).display()

In [0]:
df_a.unionByName(df_b,allowMissingColumns=True).display()

In [0]:
data1 = [(1, "Alice", 1000,'Scala'), (2, "Bob", 1500,'matlab')]
columns1 = ["id", "name", "salary","programs"]
df_a = spark.createDataFrame(data1, columns1)

data2 = [("Charlie", 3, 2000,'Java'), ("David", 4, 2500,'Python')]
columns2 = ["name", "id", "salary","programs"]
df_b = spark.createDataFrame(data2, columns2)

df_a.display()
df_b.display()

In [0]:
df_a.unionByName(df_b,allowMissingColumns=False).display()

In [0]:
df_a.unionByName(df_b,allowMissingColumns=True).display()